In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import hashlib
import math

def load_data():
    return pd.read_csv("iris.csv")

iris_data=load_data()
iris_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
species         150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [2]:
def test_set_check(identifier,test_ratio,hash):
    return bytearray(hash(np.int64(identifier)).digest())[-1]  < 51
def split_train_test_by_id(data,test_ratio,id_column,hash):
    ids=data[id_column]
    in_test_set=ids.apply(lambda id_:test_set_check(id_,test_ratio,hash))
    return in_test_set

In [3]:
iris_data=iris_data.reset_index()
in_test_set=split_train_test_by_id(iris_data,0.66,"index",hashlib.md5)
test_set=iris_data[in_test_set]
train_set=iris_data[~in_test_set]
del train_set['index']
del test_set['index']

In [4]:
train_set_array=train_set.as_matrix()
test_set_copy=test_set.copy()
del test_set['species']
test_set_array=test_set.as_matrix()

In [5]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [6]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

def summarize(dataset):
    zipd= list(zip(*dataset))
   
    del zipd[-1]
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zipd]
    return summaries


def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summary = {}
    for classValue, instances in separated.items():
       summary[classValue] = summarize(instances)
    return summary

summary = summarizeByClass(train_set_array)
print(summary)

{'setosa': [(5.002380952380952, 0.361222919645362), (3.4119047619047613, 0.39706384628363195), (1.452380952380952, 0.16854918257117807), (0.24047619047619045, 0.10605917310524214)], 'virginica': [(6.595121951219512, 0.5970557844591222), (3.012195121951219, 0.3163819858582498), (5.517073170731706, 0.4949254686437092), (2.017073170731707, 0.28098259645785023)], 'versicolor': [(5.899999999999998, 0.51720402163943), (2.7731707317073173, 0.29497829601845493), (4.246341463414634, 0.4909671888098073), (1.3195121951219513, 0.19133676096756994)]}


In [7]:
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [8]:
predictions=getPredictions(summary,test_set_array)

In [9]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [10]:
test_set_copy=test_set_copy.as_matrix()
accuracy=getAccuracy(test_set_copy,predictions)
print(accuracy)

88.46153846153845
